In [48]:
import tensorflow as tf
import os
import numpy as np

In [2]:
cwd = os.getcwd()
loadpath = cwd + '\\animal\\custom_data.npz'
loadpath
l = np.load(loadpath)
print(l.files)


['trainimg', 'imgsize', 'categories', 'trainlabel', 'testimg', 'testlabel', 'use_gray']


In [3]:
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
use_gray = l['use_gray']
if use_gray:
    nimgch = 1
else : 
    nimgch = 3
   
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim = trainimg.shape[1]
ntest = testimg.shape[0]

n_input = dim
n_channel = 64
n_classes = nclass


In [4]:
x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_classes])

stddev = 0.1
weights = {
    'c1' : tf.Variable(tf.random_normal([7,7,nimgch,n_channel], stddev=stddev)),
    'd1' : tf.Variable(tf.random_normal([n_channel*imgsize[0]*imgsize[1]//4, n_classes], stddev=stddev))#왜 //4 하지?
}

biases = {
    'c1' : tf.Variable(tf.random_normal([n_channel], stddev=stddev)),
    'd1' : tf.Variable(tf.random_normal([n_classes], stddev=stddev))
}


In [5]:
def CNN(_x, _w, _b):
    _x_r = tf.reshape(_x, shape=[-1, imgsize[0], imgsize[1], nimgch])
    _conv1 = tf.nn.conv2d(_x_r, _w['c1'], strides=[1,1,1,1], padding='SAME')
    _conv2 = tf.nn.bias_add(_conv1, _b['c1'])
    _conv3 = tf.nn.relu(_conv2)
    _pool = tf.nn.max_pool(_conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    _dense = tf.reshape(_pool, [-1, _w['d1'].get_shape().as_list()[0]])
    _logit = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
    _out = {
        'x_r':_x_r, 'conv1':_conv1, 'conv2':_conv2, 'conv3':_conv3, 'pool':_pool, 'dense':_dense, 'logit':_logit
    }
    return _out


In [35]:
cnnout = CNN(x, weights, biases)
print(y)

Tensor("Placeholder_1:0", shape=(?, 5), dtype=float32)


In [39]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=cnnout['logit']))
optm = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cost)
corr = tf.equal(tf.argmax(cnnout['logit'],1),tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, 'float'))

In [46]:
# PARAMETERS
training_epochs = 50
batch_size = 100
disp_each = 10
plot_each = 10

# LAUNCH THE GRAPH
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


In [47]:
# OPTIMIZE
for epoch in range(training_epochs):
    total_batch = int(ntrain/batch_size)
    randpermlist = np.random.permutation(ntrain)
    sum_cost = 0.
    for i in range(total_batch):
        randidx = randpermlist[i*batch_size:min((i+1)*batch_size, ntrain-1)]
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        sum_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = sum_cost / total_batch
    
    if (epoch+1) % disp_each == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost : %.9f" % (epoch+1, training_epochs, avg_cost))
        feeds = {x: batch_xs, y:batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print (" TRAIN ACCURACY : %.3f" % (train_acc))
        feeds = {x: testimg, y:testlabel}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY : %.3f" % (test_acc))
        
        

Epoch: 010/050 cost : 1.060487449
 TRAIN ACCURACY : 0.630
TEST ACCURACY : 0.557
Epoch: 020/050 cost : 0.335014611
 TRAIN ACCURACY : 0.930
TEST ACCURACY : 0.672
Epoch: 030/050 cost : 0.044633491
 TRAIN ACCURACY : 1.000
TEST ACCURACY : 0.689
Epoch: 040/050 cost : 0.016341221
 TRAIN ACCURACY : 1.000
TEST ACCURACY : 0.689
Epoch: 050/050 cost : 0.007191923
 TRAIN ACCURACY : 1.000
TEST ACCURACY : 0.672
